In [20]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
import hashlib

In [39]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
import hashlib

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()
pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)
with open('private_key_client.pem', 'wb') as f:
    f.write(pem)

pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

with open('public_key_client.pem', 'wb') as f:
    f.write(pem)

In [ ]:
with open("private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

with open("public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(
        key_file.read(),
        backend=default_backend()
    )

In [9]:
message = b'encrypt me!'
# public_key =  # Use one of the methods above to get your public key

encrypted = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
encrypted[:10]

b'>b\xdb\xeb$\xd1 \xed\x80='

In [10]:
original_message = private_key.decrypt(
    encrypted,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
original_message

b'encrypt me!'

In [19]:
# o lado de baixo:
#vamos gerar uma chave simetrica
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

import cryptography
from cryptography.fernet import Fernet

# key = Fernet.generate_key()
# print(key)

import cryptography
from cryptography.fernet import Fernet

key = Fernet.generate_key()
# print(key)

file = open('key.key', 'wb') #wb = write bytes
file.write(key)
file.close()


#  Open the file to encrypt
message = b'alalala'

fernet = Fernet(key)
encrypted = fernet.encrypt(message)
print(message)
encrypted
fernet.decrypt(encrypted)

b'alalala'


b'alalala'

In [33]:
def encrypt_with_symmetric(message, fernet_key):
    fernet = Fernet(fernet_key)
    return fernet.encrypt(message)

def decrypt_with_symmetric(encrypted_message, fernet_key):
    fernet = Fernet(fernet_key)
    return fernet.decrypt(encrypted_message)

def encrypt_with_publickey(message, public_key_path):
    with open(public_key_path, "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )
    encrypted = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
        )
    return encrypted

def decrypt_with_privatekey(encrypted_message, private_key_path):
    with open(private_key_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    original_message = private_key.decrypt(
        encrypted_message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
            )
        )
    return original_message

def sign_with_privatekey(message, private_key_path):
    '''
    Recebe uma mensagem em formato de bytes e o caminho da private key
    que será usada para assinar a mensagem
    retorna a assinatura
    '''
    with open(private_key_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
    return signature

def check_signature(signature, message, public_key_path):
    with open(public_key_path, "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )
    try: 
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        verification = True
    except: verification = False
    return verification

def resumir(mensagem):
    m = bytes(mensagem,"utf-8")
    s = hashlib.sha256()
    s.update(m)
    return s.digest()

In [34]:
signature = sign_with_privatekey(message=b'blob', private_key_path='private_key_server.pem')

In [37]:
verification = check_signature(signature=signature, message=b'blob',public_key_path='public_key_server.pem')
verification

True

In [44]:
message = b'blob'
encrypted_message = encrypt_with_publickey(message,'public_key_server.pem')
encrypted_message
# type(encrypted_message)
# decrypted_message = decrypt_with_privatekey(encrypted_message,'private_key_server.pem')
# type(decrypted_message)

b'\x05\x80C\xb6\xbfr\x05\x99M\xc9\xa5h6\x8bu\x98!G&\x81\xa2\x0b\xe8\xef\xeb\x01\xfew\x17\xe2\xf0\xf4=9o\xf4_\x9dn\x8b\xfb\xdf\x83\x07\xe0\x8a\xa4\xcb\xcdqqNl\xd6EpY\xd1\xd0\xef\x96\x8b\ns\xbf\xea/0my\xd4\xf0\n\x1c\xf1\xc9\'owD\x7f5\x1e8Y\xee\xbfXO)\xbf\xe7)\x1a\x1f\x02\xcc\xec\xc5:\x92\x13\x89\xfdV\x9b9\xec8\x99\xdb\xc2/\xbd\x7f\x82Q\x9c\xb9\x86Nc\x00T\x9c\x10\xc6\xfb\xf8K\xfe@\x1c\x1e\x01\xfd\xad\xc9\xec#\x02\xf3\x91\x18\x7f\xbe\x10\x824\xe2o\x9f\x83\xfe\xea\xba\xd06\xd5e\x02\xb0\x92\xc3*#D\x9f\xcb\xbd\xc9X\xb9B\x8d\xb9\xcb*\xd0\xb7\xcb\x94\x1ay\x82\xf2\xabFO\x1c\x8c\x1fTA\xba3"0\xf3U\x802\x00\xf2M\xbe\'\xc2u!\xad!\xbe0\xed\xdd\x12\xb5\xa9\x9b\xe7mJ`\xe9\xa9Wb\xc2\x8a1B\xe4aS\xfc\xde\xce$\x9c\xd5~\x11\x80zF\x18\x9b\x9e c\x9b3A\xee\xf1'